In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import matplotlib.pyplot as plt
import keras.backend as K
import tensorflow as tf
from tensorflow.keras.callbacks import TensorBoard
from sklearn.utils import shuffle
#import pickle5 as pickle
from sklearn.metrics import multilabel_confusion_matrix as confusion_matrix
from sklearn.metrics import accuracy_score, f1_score, precision_score, recall_score
from sklearn.externals import joblib 
from google.colab import drive
from keras.utils.np_utils import to_categorical

In [ ]:
drive.mount('/content/drive')

In [ ]:
#Importing the dataset

train_set_x = pd.read_csv(r'/content/drive/My Drive/Model_Datasets/train_traffic_x.csv', header=None)
test_set_x = pd.read_csv('/content/drive/My Drive/Model_Datasets/cropped_dataset.csv')
train_set_y = pd.read_csv(r'/content/drive/My Drive/Model_Datasets/train_traffic_y.csv', header=None)
test_set_y = pd.read_csv('/content/drive/My Drive/Model_Datasets/cropped_label.csv') 
print("Done reading .csv files")

Done reading .csv files


In [ ]:
#Splitting the dataset
X_train = train_set_x.iloc[:,:].values
y_train = train_set_y.iloc[:,-1].values

In [ ]:
# Only to use for importing test_traffix_x.csv and test_traffix_y.csv
test_dataset_y = pd.read_csv('/content/drive/My Drive/Model_Datasets/test_traffic_y.csv')
test_dataset_x = pd.read_csv('/content/drive/My Drive/Model_Datasets/test_traffic_x.csv') 
X_dataset_test = test_dataset_x.iloc[:,:].values
y_dataset_test = test_dataset_y.iloc[:,-1].values
X_dataset_test = X_dataset_test.reshape(X_dataset_test.shape[0], 32, 32, 3)
y_dataset_test = y_dataset_test.reshape(y_dataset_test.shape[0], 1)
Y_dataset_test = to_categorical( y_dataset_test, num_classes = 43)

In [ ]:
#Only to use when all_dataset.csv and all_label.csv is used
X_test = test_set_x.to_numpy()
y_test = test_set_y.to_numpy()

In [ ]:
#Shaping X data and converting Y data to a one hot vector

X_train = X_train.reshape(X_train.shape[0], 32, 32, 3)
X_test = X_test.reshape(X_test.shape[0], 32, 32, 3)
y_train = y_train.reshape(y_train.shape[0], 1)
y_test = y_test.reshape(y_test.shape[0], 1)
X_train, y_train = shuffle(X_train, y_train, random_state=0)
plt.imshow(X_train[223])
plt.show()

In [ ]:
print(X_dataset_test.shape)
print(Y_dataset_test.shape)

(12629, 32, 32, 3)
(12629, 43)


In [ ]:
#parameters specification
batch_size = 32
num_classes = 43
epochs = 50
img_size = 32

In [ ]:
#One hot encoding

Y_train = to_categorical( y_train, num_classes = 43)
Y_test = to_categorical( y_test, num_classes = 43)

In [ ]:
#Developing the CNN model
print("Devoloping a model")

model = tf.keras.models.Sequential()
model.add(tf.keras.layers.Conv2D(6, kernel_size = (5, 5), activation='relu', input_shape=(img_size, img_size, 3)))
model.add(tf.keras.layers.MaxPooling2D(pool_size=(2,2)))
model.add(tf.keras.layers.BatchNormalization())
model.add(tf.keras.layers.Conv2D(16, kernel_size = (5,5), activation='relu'))
model.add(tf.keras.layers.MaxPooling2D(pool_size=(2,2)))
model.add(tf.keras.layers.BatchNormalization())
model.add(tf.keras.layers.Flatten())
model.add(tf.keras.layers.Dense(120, activation='relu'))
model.add(tf.keras.layers.Dense(num_classes, activation='softmax'))

print("Model Developed")

Devoloping a model
Model Developed


In [ ]:
model.summary() 

In [ ]:
#Compilation

import keras
from keras.optimizers import SGD, RMSprop, Adam, Adagrad, Adadelta

opt = tf.keras.optimizers.Adagrad(lr=0.03)
model.compile(loss='categorical_crossentropy', optimizer=opt, metrics = ['accuracy'])

#model.compile(loss='categorical_crossentropy',
#              optimizer=keras.optimizers.SGD(lr=1e-3, momentum=0.9),
#              metrics=['accuracy'])

#opt = keras.optimizers.Adam(learning_rate=0.01)
#model.compile(loss='categorical_crossentropy', optimizer=opt, metrics=['accuracy'])


history = model.fit(X_train, Y_train, batch_size=128, 
          epochs=50, verbose=1,validation_split=0.1)

print("................TRAINING DONE....................")

In [ ]:
#Average Training Accuracy and Loss
history.history['accuracy']
accuracy = np.mean(history.history['accuracy'])
print('The training accuracy is', accuracy*100)
history.history['loss']
loss = np.mean(history.history['loss'])
print('The training loss is', loss)
history.history['val_accuracy']
val_accuracy = np.mean(history.history['val_accuracy'])
print('The validation accuracy is', val_accuracy*100)
history.history['val_loss']
val_loss = np.mean(history.history['val_loss'])
print('The validation loss is', val_loss)

In [ ]:
#plotting the accuracy and loss
plt.plot(history.history['accuracy'])
plt.plot(history.history['val_accuracy'])
plt.title('model accuracy')
plt.ylabel('accuracy')
plt.xlabel('epoch')
plt.legend(['train', 'validation'], loc='upper left')
plt.show()

plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.title('model loss')
plt.ylabel('loss')
plt.xlabel('epoch')
plt.legend(['train', 'validation'], loc='upper left')
plt.show()

In [ ]:
# Evaluation Metrics
target_names = []
for i in range(43):
    a = 'Object '
    b = str(i)
    c = a+b
    c = [i]
    target_names.append((a+b))

def reports(X_test,y_test):
    Y_pred = model.predict(X_test)
    y_pred = np.argmax(Y_pred, axis=1)
    classification = classification_report(np.argmax(y_test, axis=1), y_pred, target_names=target_names)
    confusion = confusion_matrix(np.argmax(y_test, axis=1), y_pred)
    score = model.evaluate(X_test, y_test, batch_size=32)
    Test_Loss = score[0]*100
    Test_accuracy = score[1]*100
    kc=cohen_kappa_score(np.argmax(y_test, axis=1), y_pred)
    return classification, confusion, Test_Loss, Test_accuracy ,kc

# Testing Dataset

In [ ]:
from sklearn.metrics import classification_report, confusion_matrix,cohen_kappa_score
from sklearn.metrics import mean_squared_error, mean_absolute_error, precision_score
classification, confusion, Test_loss, Test_accuracy,kc = reports(X_dataset_test,Y_dataset_test)
classification = str(classification)
confusion_str = str(confusion)

In [ ]:
print("confusion matrix: ")
print('{}'.format(confusion_str))
print("KAppa Coeefecient=",kc)
print('Test loss {} (%)'.format(Test_loss))
print('Test accuracy {} (%)'.format(Test_accuracy))
print(classification)

In [ ]:
# Plot Confusion Matrix
import matplotlib.pyplot as plt
import itertools
%matplotlib inline
def plot_confusion_matrix(cm, classes,
                          normalize=False,
                          title='Confusion matrix',
                          cmap=plt.get_cmap("Blues")):
    Normalized = cm.astype('float') / cm.sum(axis=1)[:, np.newaxis]
    if normalize:
        cm = Normalized
        print("Normalized confusion matrix")
    else:
        print('Confusion matrix, without normalization')
    plt.imshow(Normalized, interpolation='nearest', cmap=cmap)
    plt.colorbar()
    plt.title(title)
    tick_marks = np.arange(len(classes))
    plt.xticks(tick_marks, classes, rotation=90)
    plt.yticks(tick_marks, classes)

    fmt = '.2f' if normalize else 'd'
    for i, j in itertools.product(range(cm.shape[0]), range(cm.shape[1])):
        thresh = cm[i].max() / 2.
        plt.text(j, i, format(cm[i, j], fmt),
                 horizontalalignment="center",
                 color="white" if cm[i, j] > thresh else "black")

    plt.tight_layout()
    plt.ylabel('True label')
    plt.xlabel('Predicted label')


plt.figure(figsize=(17,17))
plot_confusion_matrix(confusion, classes=target_names, normalize=False, 
                      title='Confusion matrix, without normalization')
#plt.savefig("/content/drive/My Drive/results/InceptionResNetV2/caltech101/confusion_matrix_without_normalization.png")
plt.show()
plt.figure(figsize=(25,25))
plot_confusion_matrix(confusion, classes=target_names, normalize=True, 
                      title='Normalized confusion matrix')
#plt.savefig("/content/drive/My Drive/results/InceptionResNetV2/caltech101/confusion_matrix_with_normalization.png")
plt.show()

# Cropped Images

In [ ]:
from sklearn.metrics import classification_report, confusion_matrix,cohen_kappa_score
from sklearn.metrics import mean_squared_error, mean_absolute_error, precision_score
classification, confusion, Test_loss, Test_accuracy,kc = reports(X_test,Y_test)
classification = str(classification)
confusion_str = str(confusion)

In [ ]:
print("confusion matrix: ")
print('{}'.format(confusion_str))
print("KAppa Coeefecient=",kc)
print('Test loss {} (%)'.format(Test_loss))
print('Test accuracy {} (%)'.format(Test_accuracy))
print(classification)

In [ ]:
# Plit Confusion Matrix
import matplotlib.pyplot as plt
import itertools
%matplotlib inline
def plot_confusion_matrix(cm, classes,
                          normalize=False,
                          title='Confusion matrix',
                          cmap=plt.get_cmap("Blues")):
    Normalized = cm.astype('float') / cm.sum(axis=1)[:, np.newaxis]
    if normalize:
        cm = Normalized
        print("Normalized confusion matrix")
    else:
        print('Confusion matrix, without normalization')
    plt.imshow(Normalized, interpolation='nearest', cmap=cmap)
    plt.colorbar()
    plt.title(title)
    tick_marks = np.arange(len(classes))
    plt.xticks(tick_marks, classes, rotation=90)
    plt.yticks(tick_marks, classes)

    fmt = '.2f' if normalize else 'd'
    for i, j in itertools.product(range(cm.shape[0]), range(cm.shape[1])):
        thresh = cm[i].max() / 2.
        plt.text(j, i, format(cm[i, j], fmt),
                 horizontalalignment="center",
                 color="white" if cm[i, j] > thresh else "black")

    plt.tight_layout()
    plt.ylabel('True label')
    plt.xlabel('Predicted label')


plt.figure(figsize=(17,17))
plot_confusion_matrix(confusion, classes=target_names, normalize=False, 
                      title='Confusion matrix, without normalization')
#plt.savefig("/content/drive/My Drive/results/InceptionResNetV2/caltech101/confusion_matrix_without_normalization.png")
plt.show()
plt.figure(figsize=(25,25))
plot_confusion_matrix(confusion, classes=target_names, normalize=True, 
                      title='Normalized confusion matrix')
#plt.savefig("/content/drive/My Drive/results/InceptionResNetV2/caltech101/confusion_matrix_with_normalization.png")
plt.show()